In [7]:
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
import cv2
import pandas as pd

import os
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential, Model
from keras.layers import Dense, Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.utils import to_categorical
import keras.utils as image
from  matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
train=pd.DataFrame(columns=['file_path','class_label'])
test=pd.DataFrame(columns=['file_path','class_label'])


In [9]:
train_dir=r'D:\ml_dataset\research\Warp-C\train_all'
test_dir=r'D:\ml_dataset\research\Warp-C\test_all'

In [18]:
for class_folder in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, class_folder)):
        class_label = class_folder
        class_folder_path = os.path.join(train_dir, class_folder)
        for image_file in os.listdir(class_folder_path):
            image_file_path = os.path.join(class_folder_path, image_file)
            train= pd.concat([train, pd.DataFrame([{'file_path': image_file_path, 'class_label': class_label}])], ignore_index=True)

In [19]:
for class_folder in os.listdir(test_dir):
    if os.path.isdir(os.path.join(test_dir, class_folder)):
        class_label = class_folder
        class_folder_path = os.path.join(test_dir, class_folder)
        for image_file in os.listdir(class_folder_path):
            image_file_path = os.path.join(class_folder_path, image_file)
            test= pd.concat([test, pd.DataFrame([{'file_path': image_file_path, 'class_label': class_label}])], ignore_index=True)

In [20]:
train

,file_path,class_label
0,D:\ml_dataset\research\Warp-C\test_all\milk-ca...,milk-cardboard
1,D:\ml_dataset\research\Warp-C\train_all\bottle...,bottle-blue
2,D:\ml_dataset\research\Warp-C\train_all\bottle...,bottle-blue
3,D:\ml_dataset\research\Warp-C\train_all\bottle...,bottle-blue
4,D:\ml_dataset\research\Warp-C\train_all\bottle...,bottle-blue
...,...,...
8819,D:\ml_dataset\research\Warp-C\train_all\milk-c...,milk-cardboard
8820,D:\ml_dataset\research\Warp-C\train_all\milk-c...,milk-cardboard
8821,D:\ml_dataset\research\Warp-C\train_all\milk-c...,milk-cardboard
8822,D:\ml_dataset\research\Warp-C\train_all\milk-c...,milk-cardboard


In [21]:
test

,file_path,class_label
0,D:\ml_dataset\research\Warp-C\test_all\bottle-...,bottle-blue
1,D:\ml_dataset\research\Warp-C\test_all\bottle-...,bottle-blue
2,D:\ml_dataset\research\Warp-C\test_all\bottle-...,bottle-blue
3,D:\ml_dataset\research\Warp-C\test_all\bottle-...,bottle-blue
4,D:\ml_dataset\research\Warp-C\test_all\bottle-...,bottle-blue
...,...,...
1546,D:\ml_dataset\research\Warp-C\test_all\milk-ca...,milk-cardboard
1547,D:\ml_dataset\research\Warp-C\test_all\milk-ca...,milk-cardboard
1548,D:\ml_dataset\research\Warp-C\test_all\milk-ca...,milk-cardboard
1549,D:\ml_dataset\research\Warp-C\test_all\milk-ca...,milk-cardboard


In [26]:
train_image=[]
for i in tqdm(range(train.shape[0])):
    img=image.load_img(train['file_path'][i],target_size=(32,32,1),grayscale=False)
    img=image.img_to_array(img)
    img=img/255
    train_image.append(img)
X_train=np.array(train_image)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8824/8824 [00:17<00:00, 493.83it/s]


In [27]:
X_train

array([[[[0.10196079, 0.09803922, 0.09019608],
         [0.05882353, 0.08235294, 0.07450981],
         [0.08235294, 0.12156863, 0.09019608],
         ...,
         [0.29411766, 0.2784314 , 0.18039216],
         [0.5058824 , 0.5019608 , 0.53333336],
         [0.09019608, 0.08627451, 0.07843138]],

        [[0.23529412, 0.22352941, 0.2509804 ],
         [0.14117648, 0.14117648, 0.13333334],
         [0.10980392, 0.10980392, 0.11764706],
         ...,
         [0.41568628, 0.4117647 , 0.39215687],
         [0.35686275, 0.40784314, 0.43137255],
         [0.15686275, 0.12156863, 0.1882353 ]],

        [[0.15686275, 0.18431373, 0.20784314],
         [0.10588235, 0.07058824, 0.08235294],
         [0.3882353 , 0.34117648, 0.28627452],
         ...,
         [0.41568628, 0.39607844, 0.41960785],
         [0.19607843, 0.25882354, 0.21568628],
         [0.22352941, 0.1764706 , 0.1764706 ]],

        ...,

        [[0.18039216, 0.15294118, 0.12156863],
         [0.07843138, 0.06666667, 0.04705882]

In [28]:
X_train.shape

(8824, 32, 32, 3)

In [29]:
Y_train=train['class_label'].values

In [30]:
Y_train

array(['milk-cardboard', 'bottle-blue', 'bottle-blue', ...,
       'milk-cardboard', 'milk-cardboard', 'milk-cardboard'], dtype=object)

In [31]:
nsamples, nx, ny, nrgb = X_train.shape
X_train2 = X_train.reshape((nsamples,nx*ny*nrgb))

In [32]:
X_train2.shape

(8824, 3072)

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
model=RandomForestClassifier()

In [35]:
model.fit(X_train2,Y_train)

RandomForestClassifier()

In [36]:
test_image=[]
for i in tqdm(range(test.shape[0])):
    img=image.load_img(test['file_path'][i],target_size=(32,32,1),grayscale=False)
    img=image.img_to_array(img)
    img=img/255
    test_image.append(img)
X_test=np.array(test_image)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1551/1551 [00:43<00:00, 35.87it/s]


In [37]:
X_test.shape

(1551, 32, 32, 3)

In [38]:
Y_test=test['class_label'].values

In [39]:
nsamples, nx, ny, nrgb = X_test.shape
X_test2 = X_test.reshape((nsamples,nx*ny*nrgb))

In [40]:
X_test2.shape

(1551, 3072)

In [42]:
y_pred=model.predict(X_test2)
y_pred

array(['bottle-transp', 'bottle-dark', 'bottle-transp', ...,
       'bottle-blue', 'milk-cardboard', 'milk-cardboard'], dtype=object)

In [43]:
accuracy_score(y_pred,Y_test)
print(classification_report(y_pred,Y_test))

                         precision    recall  f1-score   support

            bottle-blue       0.25      0.23      0.24       114
       bottle-blue-full       0.07      0.75      0.13         4
          bottle-blue5l       0.10      0.50      0.16        14
     bottle-blue5l-full       0.00      0.00      0.00         0
            bottle-dark       0.86      0.40      0.54       207
       bottle-dark-full       0.06      0.17      0.09        12
           bottle-green       0.50      0.41      0.45        90
      bottle-green-full       0.12      0.40      0.18        10
            bottle-milk       0.53      0.33      0.41        91
       bottle-milk-full       0.14      0.75      0.24         4
      bottle-multicolor       0.07      0.50      0.12         4
bottle-multicolorv-full       0.33      1.00      0.50         7
             bottle-oil       0.10      1.00      0.19         5
        bottle-oil-full       0.00      0.00      0.00         0
          bottle-transp 

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
confusion_matrix(y_pred,Y_test)

array([[ 26,  14,  22,   4,   1,   0,   1,   7,   2,   0,   4,   1,   4,
          0,   7,   3,   0,   1,   5,   3,   4,   2,   1,   0,   0,   1,
          0,   1],
       [  0,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  2,   1,   7,   2,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  6,   2,   2,   0,  82,  28,   6,   2,   0,   0,   9,   1,   4,
          0,   5,   2,   3,   1,  20,   0,   4,   0,   1,   9,   6,   2,
          9,   3],
       [  0,   0,   0,   0,   0,   2,   0,   1,   0,   0,   1,   0,   0,
          0,   1,   1,   1,   0,   0,   0,   0,   1,   0,   3,   0,   1,
          0,   0],
       [  